In [46]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import denoise
from model import *
import torch.nn.functional as F

In [47]:
data = pd.read_csv("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/rssi/rssi_for_4AP.csv")

In [48]:
coordinates = {
        1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
        36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
        2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
        6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
        12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
        16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
        20: (5.4, 6.0), 21: (6.0, 6.0),
        22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
        26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
        41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
        44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
        47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
    }

In [49]:
from torch.utils.data import Dataset
import torch

class RSSIDataset(Dataset):
    def __init__(self, rssi_data, labels, coordinates_dict, augment=False, rssi_mask_prob=0.01):
        self.rssi_data = torch.tensor(rssi_data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)  # one-hot labels
        self.augment = augment
        self.rssi_mask_prob = rssi_mask_prob
        self.coordinates_dict = coordinates_dict

        # 將 class index（從 one-hot）反解為 ID（1~49）
        self.label_ids = torch.argmax(self.labels, dim=1).numpy() + 1

        # 對應到回歸座標
        self.reg_coords = torch.tensor(
            [coordinates_dict[int(lid)] for lid in self.label_ids],
            dtype=torch.float32
        )

    def __len__(self):
        return len(self.rssi_data)

    def __getitem__(self, idx):
        rssi = self.rssi_data[idx]
        label = self.labels[idx]              # one-hot
        reg_target = self.reg_coords[idx]     # 對應的 (x, y)

        if self.augment:
            mask = torch.rand(rssi.shape) < self.rssi_mask_prob
            rssi = rssi.clone()  # 避免修改原始資料
            rssi[mask] = 0.0

        return rssi, label, reg_target



In [51]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 假設你已經有 df_merged_all
rssi_columns = ['AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']

# 1. 填補 RSSI 缺值
data[rssi_columns] = data[rssi_columns].fillna(0)

# 2. Label 編碼（如果是整數，可以直接用）
labels = data['Label'].values
rssi_data = data[rssi_columns].values






In [53]:
rssi_data

array([[-60., -68., -61., -59.],
       [-62., -70., -59., -59.],
       [-61., -69., -62., -55.],
       ...,
       [-65., -65., -61., -60.],
       [-65., -65., -61., -58.],
       [-66., -64., -60., -59.]])

In [54]:
labels

array([11, 11, 11, ..., 35, 35, 35])

In [55]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(labels).reshape(-1, 1))

In [56]:
train_data, temp_data, train_labels, temp_labels = train_test_split(
    rssi_data, one_hot_labels, test_size=0.3, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(
    temp_data, temp_labels, test_size=1/3, random_state=42)

In [58]:
train_dataset = RSSIDataset(train_data, train_labels,coordinates, augment=True)
val_dataset = RSSIDataset(val_data, val_labels, coordinates)
test_dataset = RSSIDataset(test_data, test_labels, coordinates)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# for train_input, train_label in train_loader:
#     print(train_input.shape, train_label.shape)

ValueError: too many values to unpack (expected 2)

In [60]:
print(train_label[1])
argmax_label = torch.argmax(train_label[1])
print(argmax_label.item())

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
40


In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
class RSSIClassifier(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(RSSIClassifier, self).__init__()
        self.fc1 =torch.nn.Linear(input_size, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.dropout1 = torch.nn.Dropout(0.5)

        self.fc2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.dropout2 = torch.nn.Dropout(0.5)

        self.fc3 = torch.nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [24]:
model = RSSIClassifier(4, num_classes=49).to(device)

In [25]:

dummy_rssi = torch.randn(32, 4).to(device)       # RSSI 輸入: (batch=1, 4-d)
output = model(dummy_rssi)
print("Output shape:", output.shape)  # 預期: (1, 49)

# 安裝 torchinfo (如果還沒安裝)
# pip install torchinfo

from torchinfo import summary
dummy_rssi = torch.randn(1, 4).to(device)
summary(model, input_data=dummy_rssi)


Output shape: torch.Size([32, 49])


Layer (type:depth-idx)                   Output Shape              Param #
RSSIClassifier                           [1, 49]                   --
├─Linear: 1-1                            [1, 128]                  640
├─BatchNorm1d: 1-2                       [1, 128]                  256
├─Dropout: 1-3                           [1, 128]                  --
├─Linear: 1-4                            [1, 64]                   8,256
├─BatchNorm1d: 1-5                       [1, 64]                   128
├─Dropout: 1-6                           [1, 64]                   --
├─Linear: 1-7                            [1, 49]                   3,185
Total params: 12,465
Trainable params: 12,465
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05

In [26]:
import torch.nn as nn
import torch.optim as optim

# 損失函數
criterion = nn.CrossEntropyLoss()

# 優化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 學習率調整器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [27]:
print(train_loader.dataset[0][0], train_loader.dataset[0][1])

tensor([-61., -66., -56., -52.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [28]:
import torch
import matplotlib.pyplot as plt

# 儲存最佳模型相關設定
best_val_loss = float('inf')
best_model_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth"

# 訓練參數
epochs = 200

# Early Stopping 參數
patience = 20
counter = 0  

# 紀錄訓練過程中的 loss 和 accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    # ---- 訓練階段 ----
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for rssi_inputs, labels in train_loader:
        rssi_inputs = rssi_inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(rssi_inputs)

        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * rssi_inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        train_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ---- 驗證階段 ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for rssi_inputs, labels in val_loader:
            rssi_inputs = rssi_inputs.to(device)
            labels = labels.to(device)

            outputs = model(rssi_inputs)
            loss = criterion(outputs, torch.argmax(labels,dim=1))

            val_loss += loss.item() * rssi_inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            val_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # 儲存最佳模型
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"⏹️ Early stop at epoch {epoch+1}")
        break

print("✅ 訓練完成！")


Epoch [1/200] | Train Loss: 3.2215 | Train Acc: 15.11% | Val Loss: 2.2638 | Val Acc: 51.18%
✅ 儲存最佳模型 (Val Loss: 2.2638) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [2/200] | Train Loss: 2.4488 | Train Acc: 27.38% | Val Loss: 1.6515 | Val Acc: 63.86%
✅ 儲存最佳模型 (Val Loss: 1.6515) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [3/200] | Train Loss: 2.2237 | Train Acc: 32.02% | Val Loss: 1.4194 | Val Acc: 68.53%
✅ 儲存最佳模型 (Val Loss: 1.4194) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [4/200] | Train Loss: 2.1383 | Train Acc: 34.36% | Val Loss: 1.2947 | Val Acc: 70.59%
✅ 儲存最佳模型 (Val Loss: 1.2947) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth
Epoch [5/200] | Train Loss: 2.0915 | Train Acc: 35.13% | Val Loss: 1.2453 | Val Acc: 73.35%
✅ 儲存最佳模型 (Val Loss: 1.2453) 至 /media/mcs

In [62]:
# 載入最佳模型
model.load_state_dict(torch.load("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls.pth"))
model.eval()

# 測試模型
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for rssi_input, labels in test_loader:
        rssi_inputs, labels = rssi_input.to(device), labels.to(device)
        outputs = model(rssi_inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
            
        test_loss += loss.item() * rssi_inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
        # 儲存真實標籤與預測標籤
        all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_loss /= len(test_loader.dataset)
accuracy = 100 * correct / total

print(f"📊 測試損失: {test_loss:.4f}, 測試準確率: {accuracy:.2f}%")

ValueError: too many values to unpack (expected 2)

In [32]:
import numpy as np
adjusted_labels = np.array(all_labels) + 1
adjusted_predictions = np.array(all_predictions) + 1
def compute_mean_distance_error(y_true, y_pred, coordinates):
    """
    y_true, y_pred: 一維的 NumPy 陣列，分別存放真實和預測的 label（整數）
    coordinates: dict, label -> (x, y)
    """
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        # 取出對應的座標
        if true_label not in coordinates or pred_label not in coordinates:
            # 若某個 label 不在座標字典內，就跳過（或視需求處理）
            print(f"Warning: Label {true_label} or {pred_label} not in coordinates.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        # 計算歐氏距離
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors) if errors else None

COORDINATES = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

mean_error = compute_mean_distance_error(adjusted_labels, adjusted_predictions, COORDINATES)
print("Mean distance error:", mean_error)


Mean distance error: 0.6830781652838999


# 雙輸出

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RSSIClsRegNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RSSIClsRegNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.5)

        # 分支 1：分類
        self.classifier = nn.Linear(64, num_classes)

        # 分支 2：回歸 (預測 X, Y)
        self.regressor = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        cls_out = self.classifier(x)  # 分類輸出
        reg_out = self.regressor(x)   # 回歸輸出 (x, y 座標)

        return cls_out, reg_out


In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

alpha = 0.1  # 回歸損失權重
epochs = 200
best_val_loss = float('inf')
best_model_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_reg.pth"
patience = 20
counter = 0
save_enabled = True

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

# 儲存訓練記錄
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Loss 定義
criterion_cls = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

# Label ID → 座標對應函數
def labels_to_coords(label_tensor, coord_dict):
    coords = [coord_dict[int(l.item()) + 1] for l in label_tensor.cpu()]
    return torch.tensor(coords, dtype=torch.float32, device=label_tensor.device)

# 訓練迴圈
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_class_loss = 0.0
    train_reg_loss = 0.0
    train_correct = 0
    total_train = 0

    for rssi_inputs, labels, _ in train_loader:
        rssi_inputs = rssi_inputs.to(device)
        labels = labels.to(device)
        target_class = torch.argmax(labels, dim=1)

        optimizer.zero_grad()
        class_out, reg_out = model(rssi_inputs)

        loss_cls = criterion_cls(class_out, target_class)
        true_coords = labels_to_coords(target_class, COORDINATES)
        loss_reg = criterion_reg(reg_out, true_coords)
        loss = loss_cls + alpha * loss_reg

        loss.backward()
        optimizer.step()

        batch_size = rssi_inputs.size(0)
        train_loss += loss.item() * batch_size
        train_class_loss += loss_cls.item() * batch_size
        train_reg_loss += loss_reg.item() * batch_size

        _, predicted = torch.max(class_out, 1)
        train_correct += (predicted == target_class).sum().item()
        total_train += batch_size

    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_class_loss = train_class_loss / len(train_loader.dataset)
    avg_train_reg_loss = train_reg_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ----- 驗證 -----
    model.eval()
    val_loss = 0.0
    val_class_loss = 0.0
    val_reg_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for rssi_inputs, labels in val_loader:
            rssi_inputs = rssi_inputs.to(device)
            labels = labels.to(device)
            target_class = torch.argmax(labels, dim=1)

            class_out, reg_out = model(rssi_inputs)
            loss_cls = criterion_cls(class_out, target_class)
            true_coords = labels_to_coords(target_class, COORDINATES)
            loss_reg = criterion_reg(reg_out, true_coords)
            loss = loss_cls + alpha * loss_reg

            batch_size = rssi_inputs.size(0)
            val_loss += loss.item() * batch_size
            val_class_loss += loss_cls.item() * batch_size
            val_reg_loss += loss_reg.item() * batch_size

            _, predicted = torch.max(class_out, 1)
            val_correct += (predicted == target_class).sum().item()
            total_val += batch_size

    avg_val_loss = val_loss / len(val_loader.dataset)
    avg_val_class_loss = val_class_loss / len(val_loader.dataset)
    avg_val_reg_loss = val_reg_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    scheduler.step(avg_val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Train Loss: {avg_train_loss:.4f} (Cls: {avg_train_class_loss:.4f}, Reg: {avg_train_reg_loss:.4f}) | "
          f"Train Acc: {train_acc:.2f}% || "
          f"Val Loss: {avg_val_loss:.4f} (Cls: {avg_val_class_loss:.4f}, Reg: {avg_val_reg_loss:.4f}) | "
          f"Val Acc: {val_acc:.2f}%")

    # 儲存模型
    if save_enabled:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
            counter = 0
        else:
            counter += 1
            print(f"⏳ 沒改善: {counter}/{patience}")
            if counter >= patience:
                print(f"⚠️ 已達 patience={patience}，提前停止訓練")
                break

print("🎉 訓練完成！")


/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


ValueError: too many values to unpack (expected 2)